# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# import numpy and pandas
import numpy as np
import pandas as pd


# Challenge 1 - Independent Sample T-tests

In this challenge, we will be using the Pokemon dataset. Before applying statistical methods to this data, let's first examine the data.

To load the data, run the code below.

In [2]:
# Run this code:

pokemon = pd.read_csv('../pokemon.csv')

Let's start off by looking at the `head` function in the cell below.

In [4]:
# Your code here:
pokemon.head()


#                   Name Type 1  Type 2  Total  HP  Attack  Defense  \
0  1              Bulbasaur  Grass  Poison    318  45      49       49   
1  2                Ivysaur  Grass  Poison    405  60      62       63   
2  3               Venusaur  Grass  Poison    525  80      82       83   
3  3  VenusaurMega Venusaur  Grass  Poison    625  80     100      123   
4  4             Charmander   Fire     NaN    309  39      52       43   

   Sp. Atk  Sp. Def  Speed  Generation  Legendary  
0       65       65     45           1      False  
1       80       80     60           1      False  
2      100      100     80           1      False  
3      122      120     80           1      False  
4       60       50     65           1      False

The first thing we would like to do is compare the legendary Pokemon to the regular Pokemon. To do this, we should examine the data further. What is the count of legendary vs. non legendary Pokemons?

In [12]:
# Your code here:
pok=pokemon.groupby('Legendary').count()['#']


Compute the mean and standard deviation of the total points for both legendary and non-legendary Pokemon.

In [17]:
# Your code here:
pok=pokemon.groupby('Legendary').agg({'Total':['count','mean', 'std']})

pok

Total                        
          count        mean         std
Legendary                              
False       735  417.213605  106.760417
True         65  637.384615   60.937389

The computation of the mean might give us a clue regarding how the statistical test may turn out; However, it certainly does not prove whether there is a significant difference between the two groups.

In the cell below, use the `ttest_ind` function in `scipy.stats` to compare the the total points for legendary and non-legendary Pokemon. Since we do not have any information about the population, assume the variances are not equal.

In [24]:
# Your code here:
from scipy.stats import ttest_ind
ttest_ind(pokemon[pokemon['Legendary']==False]['Total'],pokemon[pokemon['Legendary']==True]['Total'])

### attention: the samples dont have the same variance (see standard deviation, as required for the ttest (according to documentation)

Ttest_indResult(statistic=-16.386116965872425, pvalue=3.0952457469652825e-52)

What do you conclude from this test? Write your conclusions below.

In [6]:
# Your conclusions here:
## the mean is not the same with more than 5 % certainity
## t stats indicate that the mean of pokemon with Legendary == False is smaller than the mean of the pokemons with Legendarys 


How about we try to compare the different types of pokemon? In the cell below, list the types of Pokemon from column `Type 1` and the count of each type.

In [35]:
# Your code here:

pok=pokemon.groupby('Type 1').agg({'Total':['count','mean', 'std']})
pok

pokemon.rename({'Type 1': 'Type_1'}, axis='columns')


import statsmodels.api as sm
from statsmodels.formula.api import ols

model = ols('Total ~ C(Type_1)', data=pokemon).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

### p-value is smaller than 0.05 (our significance level), 0-hypothesis can be rejected --> not all the Types have the same mean

sum_sq     df         F        PR(>F)
C(Type_1)  1.053322e+06   17.0  4.638767  2.077215e-09
Residual   1.044519e+07  782.0       NaN           NaN

Since water is the largest group of Pokemon, compare the mean and standard deviation of water Pokemon to all other Pokemon.

In [42]:
# Your code here:
pokemon[pokemon['Type 1']=='Water']['Total'].mean()
pokemon[pokemon['Type 1']=='Water']['Total'].std()


pokemon[pokemon['Type 1']!='Water']['Total'].mean()
pokemon[pokemon['Type 1']!='Water']['Total'].std()


121.09168230208066

Perform a hypothesis test comparing the mean of total points for water Pokemon to all non-water Pokemon. Assume the variances are equal. 

In [43]:
# Your code here:
ttest_ind(pokemon[pokemon['Type 1']=='Water']['Total'],pokemon[pokemon['Type 1']!='Water']['Total'])


Ttest_indResult(statistic=-0.4418547448849676, pvalue=0.6587140317488793)

Write your conclusion below.

In [10]:
# Your conclusions here:

## p-value is 0.659, 0-hypothesis can not be rejected --> the mean of the pokemons with type 1 = Water is maybe the same as the mean of all other pokemons

# Challenge 2 - Matched Pairs Test

In this challenge we will compare dependent samples of data describing our Pokemon. Our goal is to see whether there is a significant difference between each Pokemon's defense and attack scores. Our hypothesis is that the defense and attack scores are equal. In the cell below, import the `ttest_rel` function from `scipy.stats` and compare the two columns to see if there is a statistically significant difference between them.

In [47]:
# Your code here:
from scipy.stats import ttest_rel
ttest_rel(pokemon['Attack'], pokemon['Defense'])

## we already did this in previous lab....

Ttest_relResult(statistic=4.325566393330478, pvalue=1.7140303479358558e-05)

Describe the results of the test in the cell below.

In [48]:
# Your conclusions here:


## Between Attack and Defense there are significant differences.

We are also curious about whether therer is a significant difference between the mean of special defense and the mean of special attack. Perform the hypothesis test in the cell below. 

In [50]:
# Your code here:

print(round(ttest_rel(pokemon['Sp. Atk'], pokemon['Sp. Def'])[1]*100, 3))

## we already did this in previous lab....

39.337


Describe the results of the test in the cell below.

In [14]:
# Your conclusions here:

## Between Sp.Atk and Sp.Def there are no significant differences.

As you may recall, a two sample matched pairs test can also be expressed as a one sample test of the difference between the two dependent columns.

Import the `ttest_1samp` function and perform a one sample t-test of the difference between defense and attack. Test the hypothesis that the difference between the means is zero. Confirm that the results of the test are the same.

In [59]:
# Your code here:
from scipy.stats import ttest_1samp

from scipy.stats import ttest_1samp
mean=pokemon['Defense'].mean()

ttest_1samp(pokemon['Attack'], mean)

mean=pokemon['Attack'].mean()
ttest_1samp(pokemon['Defense'], mean)    

Ttest_1sampResult(statistic=-4.679124590910434, pvalue=3.3829080435248473e-06)

# Bonus Challenge - The Chi-Square Test

The Chi-Square test is used to determine whether there is a statistically significant difference in frequencies. In other words, we are testing whether there is a relationship between categorical variables or rather when the variables are independent. This test is an alternative to Fisher's exact test and is used in scenarios where sample sizes are larger. However, with a large enough sample size, both tests produce similar results. Read more about the Chi Squared test [here](https://en.wikipedia.org/wiki/Chi-squared_test).

In the cell below, create a contingency table using `pd.crosstab` comparing whether a Pokemon is legenadary or not and whether the Type 1 of a Pokemon is water or not.

In [77]:
# Your code here:
pokemon['Water'] = pokemon['Type 1'].apply(lambda x: True if x=='Water' else False)


crosstab= pd.crosstab(pokemon['Legendary'], pokemon['Water'])
crosstab


Water      False  True 
Legendary              
False        627    108
True          61      4

Perform a chi-squared test using the `chi2_contingency` function in `scipy.stats`. You can read the documentation of the function [here](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.chi2_contingency.html).

In [79]:
# Your code here:
from scipy.stats import chi2_contingency
chi2, pvalue, dof, expected = chi2_contingency(crosstab)

print(pvalue)

expected

0.08625467249550949


array([[632.1, 102.9],
       [ 55.9,   9.1]])

Based on a 95% confidence, should we reject the null hypothesis?

In [19]:
# Your answer here:
## no, we cant, because the p value is 0.086> 0.05

# Interpretation: the  difference in frequencies is not statistically significant, --> beeing Legendary has no impact / relationship with beeing waters 
